# Measure changes in seismic velocities using a single station

In this example, we will download data from a French network. That station is located where big "bassin" were made to store water, but these were affecting the local aquifer replenishing.

Import basic modules

In [1]:
from noisepy.seis import download, cross_correlate, stack, plotting_modules
from noisepy.seis.asdfstore import ASDFRawDataStore, ASDFCCStore
from noisepy.seis.datatypes import ConfigParameters
from dateutil.parser import isoparse
from noisepy.seis import noise_module
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import logging
from obspy.signal.filter import bandpass


logger = logging.getLogger(__name__)
path = "~/data" # for use in Colab
# path = os.path.expanduser("~/data") # for local runs
# print(path)

MAX_MEM = 4.0

os.makedirs(path,exist_ok=True)

raw_data_path = os.path.join(path, "RAW_DATA")
cc_data_path = os.path.join(path, "CCF")
stack_data_path = os.path.join(path, "STACK")
os.system(f"rm -rf {raw_data_path}")

0

## Ambient noise project configuration

Some key parameters

In [ ]:
config = ConfigParameters() # default config parameters which can be customized
config.inc_hours = 12
config.start_date =  isoparse("2022-06-18")
config.end_date = isoparse("2022-06-21")
config.samp_freq= 40  # (int) Sampling rate in Hz of desired processing (it can be different than the data sampling rate)
config.cc_len= 3600.0  # (float) basic unit of data length for fft (sec)
    # criteria for data selection
config.ncomp = 3  # 1 or 3 component data (needed to decide whether do rotation)


config.acorr_only = True  # only perform auto-correlation or not
config.xcorr_only = False  # only perform cross-correlation or not


config.lamin = 40       # min latitude
config.lamax = 50       # max latitude
config.lomin = 0     # min longitude
config.lomax = 10     # max longitude
config.net_list = ["FR"] # look for all network codes
config.stations = ["CHIF"]


 # pre-processing parameters
config.step= 1800.0  # (float) overlapping between each cc_len (sec)
config.stationxml= False  # station.XML file used to remove instrument response for SAC/miniseed data
config.rm_resp= "inv"  # select 'no' to not remove response and use 'inv' if you use the stationXML,'spectrum',
config.freqmin = 0.05
config.freqmax = 10.0
config.max_over_std  = 10  # threshold to remove window of bad signals: set it to 10*9 if prefer not to remove them

# TEMPORAL and SPECTRAL NORMALISATION
config.freq_norm= "rma"  # choose between "rma" for a soft whitenning or "no" for no whitening. Pure whitening is not implemented correctly at this point.
config.smoothspect_N = 10  # moving window length to smooth spectrum amplitude (points)
    # here, choose smoothspect_N for the case of a strict whitening (e.g., phase_only)

config.time_norm = "no"  # 'no' for no normalization, or 'rma', 'one_bit' for normalization in time domain,
    # TODO: change time_norm option from "no" to "None"
config.smooth_N= 10  # moving window length for time domain normalization if selected (points)

config.cc_method= "xcorr"  # 'xcorr' for pure cross correlation OR 'deconv' for deconvolution;
    # FOR "COHERENCY" PLEASE set freq_norm to "rma", time_norm to "no" and cc_method to "xcorr"

# OUTPUTS:
config.substack = True  # True = smaller stacks within the time chunk. False: it will stack over inc_hours
config.substack_len = config.cc_len  # how long to stack over (for monitoring purpose): need to be multiples of cc_len
    # if substack=True, substack_len=2*cc_len, then you pre-stack every 2 correlation windows.
    # for instance: substack=True, substack_len=cc_len means that you keep ALL of the correlations

config.maxlag= 200  # lags of cross-correlation to save (sec)
config.substack = True

## Download data

In [2]:
from obspy.clients.fdsn.client import Client
from obspy.core.utcdatetime import UTCDateTime

client = Client("RESIF")
starttime = UTCDateTime("2023-06-18T00:00:00.000000")
endtime   = UTCDateTime("2023-06-20T00:00:00.000000")
s = client.get_waveforms("FR", "CHIF", location = "*", channel = "HH*",
                         starttime = starttime, endtime = endtime)

### Download using noisepy:

(buggy for now?)

In [ ]:
# Download data locally. Enters raw data path, channel types, stations, config, and fdsn server.
download(raw_data_path, config, "RESIF")